In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

import logging

logger = logging.getLogger('')
logger.setLevel(logging.INFO)

from helper import Camera
import pickle
import scipy


In [2]:
const_rl_ddl = 200
const_warm_up = 2
const_final_round  = const_rl_ddl - 1
def spearmanFromActProb(rank, act_prob):
    x = [act_prob[i] for i in rank]
    y = [c for c in range(len(rank))]
    return scipy.stats.spearmanr(x,y)[0]
def spearmanSequenceFromActProbList(rank, act_probList):
    cos = []
    for act_prob in act_probList:
        cos.append(spearmanFromActProb(rank, act_prob))
    return np.array(cos)
def evaluateContributionsFromLogs(logs, client_nums):
    contirbution = []
    ddl = len(logs)
    for r in range(ddl):
        contirbution.append([0 for c in range(client_nums)])
    pprev_acc = logs[0].acc
    prev_participants = logs[1].participants
    prev_acc = logs[1].acc
    for r in range(2, ddl):
        for c in range(client_nums):
            contirbution[r][c] = contirbution[r-1][c]
        participants, acc = logs[r].participants, logs[r].acc
        if acc - prev_acc > prev_acc - pprev_acc:
            for c in prev_participants:
                contirbution[r][c] -= 1
            for c in participants:
                contirbution[r][c] += 1
        
        if acc < prev_acc:
            for c in participants:
                contirbution[r][c] -= 1
    return contirbution

In [3]:
# MLP         MNIST/FMNIST    5/25/100  seed=2-5    =2*3*4 = 24
# CNN         CIFAR10         5/25/100  seed=2-5    =1*3*4 = 12
# RESNET18    CIFAR100        5/25/100  seed=2-5    =1*3*4 = 12

In [4]:
def spearmanOfRoundFromCamera(camera, curr_round):
    act_probs = camera.act_probLogs[curr_round]
    contributions = evaluateContributionsFromLogs(camera.getRpmLogs(curr_round), camera.client_nums)
    sco_rl = spearmanSequenceFromActProbList(camera.rank, act_probs)
    sco_qi = spearmanSequenceFromActProbList(camera.rank, contributions[const_warm_up:])
    return sco_rl, sco_qi

In [8]:
with open('camera/cn25_pn5_dFMNIST_aMLP_plabel-skew_seed4.camera', 'rb') as f:
    camera = pickle.load(f)
curr_round = 4
sco_rl, sco_qi = spearmanOfRoundFromCamera(camera, curr_round)
# 合法轮数: const_warm_up 到 const_final_round
final_spearman_co_rl = sco_rl[const_final_round]
final_spearman_co_qi = sco_qi[const_final_round]

final_spearman_co_rl, final_spearman_co_qi

(0.7507692307692307, 0.24086931611304635)